In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

import pickle
import pandas as pd
import time

from utils import greedy, simplex_projection
from continuous import solve_continuous_quadratic

In [ ]:
setting = 'n128'
ab_list = [(1,10), (1,50), (5,10), (5,50), (10,10), (10,50), (20,10), (20,50)]

results = []
for alpha, beta in ab_list:
    params = np.load(f'data/{setting}/o{alpha}_c{beta}/params.npz')
    n = params['n']
    R = params['R']

    for trial in range(10):
        yinit = np.ones(n)*R/n
        y = np.ones(n)*R/n
        pred = np.ones(n)*R/n

        for t in range(100):
            with open(f'data/{setting}/o{alpha}_c{beta}/{trial*100+t}.pkl', 'rb') as f:
                G = pickle.load(f)

            start = time.time()
            itr_c, _ = greedy(yinit, G)
            time_c = time.time() - start

            start = time.time()
            itr_p, x = greedy(pred, G)
            time_p = time.time() - start
            
            qsol, time_qc = solve_continuous_quadratic(G)
            start = time.time()
            itr_q, _ = greedy(qsol, G)
            time_qg = time.time() - start
            time_q = time_qc + time_qg

            results.append([alpha, beta, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg])

            y += .01 * (R/np.sqrt(n)) * np.sign(x - y)
            y = simplex_projection(y, R)
            
            pred = (pred*(t+1) + y)/(t+2)
            print(alpha, beta, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg)

# create a DataFrame from the results list
df = pd.DataFrame(results, columns=['alpha', 'beta', 'trial', 't', 'itr_c', 'itr_p', 'itr_q', 'time_c', 'time_p', 'time_q', 'time_qc', 'time_qg'])
#save df as pickle in result folder
df.to_pickle(f'result/{setting}.pkl')